In [52]:
!pip install py2neo

In [54]:
from py2neo import Graph, Node, Relationship, NodeMatcher

# Connexion à Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "toure21diadie"))
matcher = NodeMatcher(graph)

# Supposons que "data" contient tes articles extraits
for article in data:
    # Nettoyage et vérification des champs essentiels
    paper_id = article.get('paper_id', '').strip() or None
    title = article.get('title', '').strip()
    abstract = article.get('abstract', '').strip()
    doi = article.get('doi', '').strip()
    
    # Création ou récupération du noeud Article
    if paper_id:
        article_node = matcher.match("Article", paper_id=paper_id).first()
        if not article_node:
            article_node = Node("Article", paper_id=paper_id, title=title, abstract=abstract, doi=doi)
            graph.create(article_node)
    else:
        article_node = Node("Article", title=title, abstract=abstract, doi=doi)
        graph.create(article_node)
    
    # Traiter les auteurs
    for author in article.get('authors', []):
        first = (author.get('first') or '').strip()
        middle = (author.get('middle') or '').strip()
        last = (author.get('last') or '').strip()
        institution = (author.get('institution') or '').strip()
        location = (author.get('location') or '').strip()
        
        if last:  # On considère que "last" est obligatoire pour identifier un auteur
            author_node = matcher.match("Auteur", last=last).first()
            if not author_node:
                author_node = Node("Auteur", first=first, middle=middle, last=last, institution=institution, location=location)
                graph.create(author_node)
            
            # Relation Article -> Auteur
            relation = Relationship(article_node, "ÉCRIT_PAR", author_node)
            graph.merge(relation)
            
            # Lier l'auteur à son institution
            if institution:
                institution_node = matcher.match("Institution", name=institution).first()
                if not institution_node:
                    institution_node = Node("Institution", name=institution, location=location)
                    graph.create(institution_node)
                
                institution_relation = Relationship(author_node, "APPARTIENT_A", institution_node)
                graph.merge(institution_relation)
    
    # Traiter les mots-clés
    for keyword in article.get('keywords', []):
        keyword_clean = keyword.strip()
        if keyword_clean:
            keyword_node = matcher.match("MotClé", keyword=keyword_clean).first()
            if not keyword_node:
                keyword_node = Node("MotClé", keyword=keyword_clean)
                graph.create(keyword_node)
            
            keyword_relation = Relationship(article_node, "A_POUR_MOT_CLÉ", keyword_node)
            graph.merge(keyword_relation)
